PDF_경우

In [ ]:
!pip install PyMuPDF

!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 677.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
question = 'Tell me what type of data is used in this sentence.'  # 질문을 입력하세요
pdf_path = '/content/sample_data/paper/journal.pone.0276781.pdf'  # PDF 문서 경로를 입력하세요
file_path2 = '/content/sample_data/paper/journal.pone.0276781.txt'  # 텍스트 파일 경로

1. Bert

In [ ]:
import fitz  # PyMuPDF
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# 1. PDF 문서에서 텍스트 추출
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text

# 2. mBERT 모델 초기화
def initialize_model():
    model_name = "bert-base-multilingual-cased"  # mBERT 모델
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForQuestionAnswering.from_pretrained(model_name)
    return tokenizer, model

# 3. 프롬프트 엔지니어링 및 질문에 대한 응답 생성
# 3. 프롬프트 엔지니어링 및 질문에 대한 응답 생성
def answer_question(question, context, tokenizer, model, max_length=512): # add max_length
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt',
                                   max_length=max_length, truncation=True) # add truncation
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer_tokens = input_ids[0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# 예제 사용

context = extract_text_from_pdf(pdf_path)

tokenizer, model = initialize_model()

answer = answer_question(question, context, tokenizer, model)

print(f'질문: {question}')
print(f'답변: {answer}')

ModuleNotFoundError: No module named 'fitz'

2. LongformerTokenizer

In [ ]:
import fitz  # PyMuPDF
from transformers import LongformerTokenizer, LongformerForQuestionAnswering
import torch

# 1. PDF 문서에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# 2. 질문에 대한 답변을 추출하는 함수
def answer_question(context, question, model, tokenizer):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', max_length=4096, truncation=True)
    input_ids = inputs['input_ids'].tolist()[0]
    attention_mask = inputs['attention_mask']

    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1

    answer_tokens = input_ids[start_index:end_index]
    answer = tokenizer.decode(answer_tokens)
    return answer


# 4. PDF에서 텍스트 추출
context = extract_text_from_pdf(pdf_path)

# 5. Longformer 모델과 토크나이저 로드
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerForQuestionAnswering.from_pretrained('allenai/longformer-base-4096')


# 7. 질문에 대한 답변 추출
answer = answer_question(context, question, model, tokenizer)
print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LongformerForQuestionAnswering were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Answer:  or ARBs on a range of
COVID-19 outcomes, including severe disease and concluded that drugs targeting the RAAS
should continue to be used [14, 15]. Moreover, in observational studies focusing on hyperten-
sive populations, no evidence was found of any worsening effect of ACEi or ARBs on out-
comes of COVID-19, and that they may even be protective. However, the majority of these
studies compared individuals taking ACEi/ARBs with those not on any antihypertensive medi-
cations [16–20]. Nevertheless, the BRACE-CORONA trial, which was the first randomised
controlled trial examining the use of ACEi and ARBs in COVID-19 patients concluded that
cardiac patients hospitalised with COVID-19 can safely continue taking these drugs [21]. The
aim of this study was to estimate the risk of severe COVID-19 between individuals with and
without hypertension and explore whether any increased risk of severe COVID-19 was depen-
dent on the absolute BP and/or type of antihypertensive treatment.
Metho

In [ ]:
from transformers import LongformerTokenizer, LongformerForQuestionAnswering
import torch

# 1. 텍스트 파일에서 텍스트 읽기 함수
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# 2. Longformer 모델 초기화 함수
def initialize_model():
    model_name = "allenai/longformer-base-4096"  # Longformer 모델
    tokenizer = LongformerTokenizer.from_pretrained(model_name)
    model = LongformerForQuestionAnswering.from_pretrained(model_name)
    return tokenizer, model

# 3. 질문에 대한 응답 생성 함수
def answer_question(question, context, tokenizer, model, max_length=4096):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt',
                                   max_length=max_length, truncation=True, padding=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Longformer에서는 `attention_mask`가 필요합니다
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer_tokens = input_ids[0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# 예제 사용
#file_path2 = '/content/sample_data/paper/journal.pone.0276781.txt'  # 텍스트 파일 경로
question = 'What do you use your data for for your experiments?'  # 질문

# 텍스트 파일에서 텍스트 읽기
context = read_text_file(file_path2)

# 모델과 토크나이저 초기화
tokenizer, model = initialize_model()

# 질문에 대한 답변 생성
answer = answer_question(question, context, tokenizer, model)

print(f'질문: {question}')
print(f'답변: {answer}')


Some weights of LongformerForQuestionAnswering were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


질문: What do you use your data for for your experiments?
답변:  was dichotomised into white and non-white (due to
small numbers) and sex was taken from the baseline UK Biobank visit. All data were assumed
to be true at the analysis baseline. Comorbidity was defined as any of the following: peripheral
vascular disease (PVD), myocardial infarction (MI), coronary heart disease (CHD) (excluding
hypertension), angina, heart failure (HF) and arrhythmia or stroke at any one of the UK Biobank follow-up visits, prevalent at baseline or history of a CV comorbidity based on International Classification of Diseases 10 (ICD10) codes from hospital inpatient episodes or from
primary health care records using primary or secondary positions of the diagnostic codes.
Death due to COVID-19 was obtained using ICD10 codes from the death records, primary
care records and hospital inpatient records. Further information including ICD10 code lists is
included in the supplement.
Medications were obtained from the U

3. BigBird_안됨


In [ ]:
import fitz  # PyMuPDF
from transformers import BigBirdTokenizer, BigBirdForQuestionAnswering
import torch

# 1. PDF 문서에서 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# 2. BigBird 모델 초기화 함수
def initialize_model():
    model_name = "google/bigbird-roberta-base"  # BigBird 모델
    tokenizer = BigBirdTokenizer.from_pretrained(model_name)
    model = BigBirdForQuestionAnswering.from_pretrained(model_name)
    return tokenizer, model

# 3. 질문에 대한 응답 생성 함수
def answer_question(question, context, tokenizer, model, max_length=4096):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt',
                                   max_length=max_length, truncation=True, padding=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer_tokens = input_ids[0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# 예제 사용
#pdf_path = '/content/sample_data/paper/journal.pone.0276781.pdf'  # PDF 문서 경로
#question = 'Please provide detailed information on the data used in the study.'  # 질문

# PDF에서 텍스트 추출
context = extract_text_from_pdf(pdf_path)

# 모델과 토크나이저 초기화
tokenizer, model = initialize_model()

# 질문에 대한 답변 생성
answer = answer_question(question, context, tokenizer, model)

print(f'질문: {question}')
print(f'답변: {answer}')


Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['qa_classifier.intermediate.dense.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


질문: Tell me what type of data is used in this sentence.
답변: 


4. openAI

In [ ]:
import fitz  # PyMuPDF
import openai

# OpenAI API 키 설정 ##
openai.api_key = 'your-openai-api-key'

def extract_text_from_pdf(pdf_path):
    """
    PDF 문서에서 텍스트를 추출하는 함수
    """
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

def get_gpt3_answer(prompt):
    """
    GPT-3 모델을 사용하여 프롬프트에 대한 답변을 생성하는 함수
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # GPT-3.5 turbo 모델 사용
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,  # 응답 길이 제한
        n=1,  # 응답 개수
        stop=None,
        temperature=0.5  # 생성된 텍스트의 다양성
    )
    return response.choices[0].message['content'].strip()

# PDF 파일 경로
pdf_path = '/content/sample_data/paper/journal.pone.0276781.pdf'

# PDF에서 텍스트 추출
text = extract_text_from_pdf(pdf_path)

# GPT-3 프롬프트 구성
question = "What's the title?"
prompt = f"Context: {text}\n\nQuestion: {question}\nAnswer:"

# GPT-3 모델에 질문하여 답변 추출
answer = get_gpt3_answer(prompt)
print(f"Answer: {answer}")

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
